# Test del StT de Watson de IBM

kernel: python 3.10.12


In [1]:
import sounddevice as sd
import numpy as np
import queue
import pyaudio
import wave
#import webrtcvad
from os.path import join, dirname
import json
import io
from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator


In [13]:
# Configuración de PyAudio
#CHUNK = 4096    #1024              # Tamaño de cada fragmento de audio
FORMAT = pyaudio.paInt16  # Formato de audio
CHANNELS = 4              # Audio mono
RATE = 16000 #11025              # Frecuencia de muestreo compatible con Whisper
DURATION = 0.03
BYTES_PRE_SAMPLE = 1
CHUNK = DURATION * RATE  
CHUNK = int(CHUNK)
# === CONFIGURACIÓN DE AUDIO ===
#CHUNK = 1024                # 64ms
FORMAT = pyaudio.paInt16


In [3]:
apikey = "GcPMqCjN5je8m_Ef62KZNEm2xjnuyWaIEBtGuN-bFdvk"
url = "https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/09491bf9-4163-452c-9671-35a92c0ff521"

API_KEY = apikey
SERVICE_URL = url

authenticator = IAMAuthenticator(apikey)
speech_to_text = SpeechToTextV1(
    authenticator=authenticator
)

speech_to_text.set_service_url(url)

#speech_models = speech_to_text.list_models().get_result()
#print(json.dumps(speech_models, indent=2))

speech_model = speech_to_text.get_model('es-ES').get_result()
print(json.dumps(speech_model, indent=2))



{
  "name": "es-ES",
  "rate": 8000,
  "language": "es-ES",
  "description": "Large Castilian Spanish Model",
  "supported_features": {
    "custom_acoustic_model": false,
    "custom_language_model": true,
    "low_latency": false,
    "speaker_labels": true
  },
  "decoder_method": "decode_ctc_v2_0_0",
  "url": "https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/09491bf9-4163-452c-9671-35a92c0ff521/v1/models/es-ES"
}


In [4]:
# Función para reproducir audio
def play_audio(dev, data):
    stream = dev.open(format=FORMAT, channels=CHANNELS, rate=RATE, output=True)
    stream.write(data)
    stream.stop_stream()
    stream.close()

In [5]:

def play_buffer(frames, rate=16000):
    """Reproduce el audio almacenado en un buffer de frames (lista de bytes)."""
    if not frames:
        print("⚠️ No hay audio para reproducir.")
        return

    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=rate,
                    output=True)

    print("🔈 Reproduciendo utterance...")
    for chunk in frames:
        stream.write(chunk)

    stream.stop_stream()
    stream.close()
    p.terminate()
    print("✅ Reproducción finalizada.")

In [6]:

def frames_to_wav_bytes(frames, rate=16000):
    buf = io.BytesIO()
    wf = wave.open(buf, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(2)  # 16 bits
    wf.setframerate(rate)
    wf.writeframes(b''.join(frames))
    wf.close()
    buf.seek(0)
    return buf


In [7]:
THRESHOLD = 0.006
def is_speech(data):
    audio = np.frombuffer(data, dtype=np.int16).astype(np.float32)
    rms_val = np.sqrt(np.mean(np.square(audio))) / 32768.0
#    print(rms_val)
    return rms_val > THRESHOLD

In [8]:
# mediante WebSockets (no funciona todavía)
# Obtenemos el token de autenticación temporal
token = speech_to_text.get_token()

# URL del WebSocket (usando el modelo español)
model = "es-ES_BroadbandModel"
ws_url = f"wss://api.eu-gb.speech-to-text.watson.cloud.ibm.com/v1/recognize?model={model}"



p = pyaudio.PyAudio()

# === FUNCIÓN DE STREAMING ===
def stream_audio(ws):
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE,
                    input=True, frames_per_buffer=CHUNK)
    print("🎙️ Hablando... (Ctrl+C para detener)")
    try:
        while True:
            data = stream.read(CHUNK, exception_on_overflow=False)
            audio64 = base64.b64encode(data).decode('utf-8')
            message = json.dumps({"audio": audio64, "action": "continue"})
            ws.send(message)
    except KeyboardInterrupt:
        print("\n🛑 Terminando transmisión...")
        stream.stop_stream()
        stream.close()
        ws.send(json.dumps({"action": "stop"}))

# === CALLBACKS DEL WEBSOCKET ===
def on_open(ws):
    print("✅ Conectado al servicio de Watson STT.")
    # Configuración inicial
    init_msg = {
        "action": "start",
        "content-type": "audio/l16;rate=16000;channels=1",
        "interim_results": True,
        "inactivity_timeout": -1,  # nunca cerrar por inactividad
        "smart_formatting": True,
    }
    ws.send(json.dumps(init_msg))

    # Lanzar hilo de audio
    threading.Thread(target=stream_audio, args=(ws,), daemon=True).start()

def on_message(ws, message):
    data = json.loads(message)
    if "results" in data:
        result = data["results"][0]
        transcript = result["alternatives"][0]["transcript"].strip()
        if result.get("final"):
            print(f"🗣️ [FINAL]: {transcript}")
        else:
            print(f"💬 [PARCIAL]: {transcript}", end='\r')

def on_error(ws, error):
    print(f"❌ Error: {error}")

def on_close(ws, code, msg):
    print("🔒 Conexión cerrada.")

# === INICIAR WEBSOCKET ===
headers = [f"Authorization: Bearer {token}"]
ws = WebSocketApp(ws_url, header=headers,
                  on_open=on_open,
                  on_message=on_message,
                  on_error=on_error,
                  on_close=on_close)

ws.run_forever()

AttributeError: 'SpeechToTextV1Adapter' object has no attribute 'get_token'

In [41]:
# Lee audio durante RECORD_SECONDS y lo guarda en un ficher OUTPUT_WAV para StT

RECORD_SECONDS = 5     # tiempo de grabación
OUTPUT_WAV = "output.wav"

p = pyaudio.PyAudio()

# === ABRIR STREAM ===
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("🎙️ Grabando...")

frames = []
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK, exception_on_overflow=False)
    frames.append(data)

print("✅ Grabación finalizada.")

stream.stop_stream()
stream.close()
p.terminate()

# === GUARDAR EN WAV ===
wf = wave.open(OUTPUT_WAV, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

print(f"💾 Audio guardado en {OUTPUT_WAV}")

# === ENVIAR A WATSON STT ===
with open(OUTPUT_WAV, 'rb') as audio_file:
    result = speech_to_text.recognize(
        audio=audio_file,
        content_type='audio/wav',
        model='es-ES_Multimedia',  # puedes cambiar por 'es-ES_BroadbandModel'
        timestamps=True,
        word_confidence=True
    ).get_result()

# === MOSTRAR RESULTADO ===
if 'results' in result and len(result['results']) > 0:
    text = result['results'][0]['alternatives'][0]['transcript']
    print(f"\n🗣️ Transcripción: {text.strip()}")
else:
    print("\n⚠️ No se detectó texto.")


🎙️ Grabando...
✅ Grabación finalizada.
💾 Audio guardado en output.wav

🗣️ Transcripción: esto es una frase de cinco segundos


In [9]:
def recognize_buffer(frames):
    """Envía los frames acumulados al STT y devuelve la transcripción."""
    if not frames:
        return None
    if len(b''.join(frames)) % 2 != 0:
        frames[-1] = frames[-1][:-1]
#    audio_bytes = b''.join(frames)
#    audio_stream = io.BytesIO(audio_bytes)
    audio_stream = frames_to_wav_bytes(frames)
    result = speech_to_text.recognize(
        audio = audio_stream,
        content_type = 'audio/wav',
        model='es-ES_BroadbandModel',
        smart_formatting=True
    ).get_result()


    if result.get('results'):
        return result['results'][0]['alternatives'][0]['transcript'].strip()
    return None



In [14]:
# VAD con buffer dinamico
# ==== INICIALIZACIÓN ====
SILENCE_LIMIT = 0.8

p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("🎙️ Esperando voz (Ctrl+C para salir)...")

frames = []
silence_counter = 0
recording = False

try:
    while True:
        data = stream.read(CHUNK, exception_on_overflow=False)

        if is_speech(data):
            if not recording:
                print("🎤 Detectada voz, grabando utterance...")
                frames = []
                recording = True
                silence_counter = 0
            frames.append(data)
        else:
            if recording:
                silence_counter += CHUNK / RATE
                if silence_counter > SILENCE_LIMIT:
                    print("🛑 Fin de utterance. Enviando a Watson...")
                    recording = False
                    silence_counter = 0
#                    play_buffer(frames, rate=RATE)
                    text = recognize_buffer(frames)
                    if text:
                        print(f"🗣️ Transcripción: {text}")
                    else:
                        print("⚠️ No se reconoció texto.")
            # si no hay voz y no estamos grabando, no hacemos nada

except KeyboardInterrupt:
    print("\n👋 Finalizando...")
    stream.stop_stream()
    stream.close()
    p.terminate()

🎙️ Esperando voz (Ctrl+C para salir)...
🎤 Detectada voz, grabando utterance...
🛑 Fin de utterance. Enviando a Watson...
🗣️ Transcripción: los
🎤 Detectada voz, grabando utterance...
🛑 Fin de utterance. Enviando a Watson...
🗣️ Transcripción: cómo que
🎤 Detectada voz, grabando utterance...
🛑 Fin de utterance. Enviando a Watson...
🗣️ Transcripción: en 1000

👋 Finalizando...


In [44]:
#VAD con fichero
# ==== INICIALIZACIÓN ====
SILENCE_LIMIT = 0.8

p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("🎙️ Esperando voz (Ctrl+C para salir)...")

frames = []
silence_counter = 0
recording = False

try:
    while True:
        data = stream.read(CHUNK, exception_on_overflow=False)

        if is_speech(data):
            if not recording:
                print("🎤 Detectada voz, grabando utterance...")
                frames = []
                recording = True
                silence_counter = 0
            frames.append(data)
        else:
            if recording:
                silence_counter += CHUNK / RATE
                if silence_counter > SILENCE_LIMIT:
                    print("🛑 Fin de utterance. Enviando a Watson...")
                    recording = False
                    silence_counter = 0
                    wf = wave.open(OUTPUT_WAV, 'wb')
                    wf.setnchannels(CHANNELS)
                    wf.setsampwidth(p.get_sample_size(FORMAT))
                    wf.setframerate(RATE)
                    wf.writeframes(b''.join(frames))
                    wf.close()

                    print(f"💾 Audio guardado en {OUTPUT_WAV}")
#                    play_buffer(frames, rate=RATE)
#                    play_audio()
                    with open(OUTPUT_WAV, 'rb') as audio_file:
                        result = speech_to_text.recognize(
                            audio=audio_file,
                            content_type='audio/wav',
                            model='es-ES_Multimedia',  # puedes cambiar por 'es-ES_BroadbandModel'
                            timestamps=True,
                            word_confidence=True
                        ).get_result()
#                    text = recognize_buffer(frames)
                    if result.get('results'):
                        text = result['results'][0]['alternatives'][0]['transcript'].strip()
                    else: 
                        text = None
                    if text:
                        print(f"🗣️ Transcripción: {text}")
                    else:
                        print("⚠️ No se reconoció texto.")
            # si no hay voz y no estamos grabando, no hacemos nada

except KeyboardInterrupt:
    print("\n👋 Finalizando...")
    stream.stop_stream()
    stream.close()
    p.terminate()

🎙️ Esperando voz (Ctrl+C para salir)...
🎤 Detectada voz, grabando utterance...
🛑 Fin de utterance. Enviando a Watson...
💾 Audio guardado en output.wav
🗣️ Transcripción: esto es una frase que duda en concreto para lo que tuvieron hablar

👋 Finalizando...
